# simple LSTM

so basic it runs on pumpkin spice

## echo sequence prediction problem

#### generating data

our echo sequence prediction problem needs data: specifically vectors of random sequences. let's use integers, and define our problem space as integers between 0 and 99.

we'll use the ```randint()``` function from the python 3 ```random``` [module](https://docs.python.org/3/library/random.html "python 3 random module docs") to generate random integers within the range we specify (in this case, 0 to 99). 

we can use the ```randint()``` function within a function of our own to generate sequences of random integers--this will be the data for our problem.

In [7]:
# randint() is inside the python random module

import random

In [9]:
# use randint() to generate a random integer between 0 and 99

rand_int = random.randint(0, 99)

rand_int

60

## more information

##### python 3 random module documentation

https://docs.python.org/3/library/random.html